In [1]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [2]:
import pandas as pd

In [3]:
dtype_df_train_cleaned_treated = {
"userId" : 'string',
"history" : 'string',
"numberOfClicksHistory" : 'Float32',
"timeOnPageHistory" : 'Float32',
"scrollPercentageHistory" : 'Float32',
"userType" : 'category'
}

In [4]:
df_train_cleaned = pd.read_csv(config["DF_TRAIN_FEATURES_CSV"], dtype=dtype_df_train_cleaned_treated)
# df_train_cleaned = pd.read_csv(config["DF_TRAIN_FEATURES_CSV"], dtype=dtype_df_train_cleaned_treated, nrows=500000)
df_train_cleaned.drop(columns=["Unnamed: 0"],inplace=True)
df_train_cleaned.head(2)

,userId,history,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,userType
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,0.0,0.292439,0.8158,Non-Logged
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,0.0,0.48115,0.7336,Non-Logged


In [5]:
df_train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6349896 entries, 0 to 6349895
Data columns (total 6 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   userId                   string  
 1   history                  string  
 2   numberOfClicksHistory    Float32 
 3   timeOnPageHistory        Float32 
 4   scrollPercentageHistory  Float32 
 5   userType                 category
dtypes: Float32(3), category(1), string(2)
memory usage: 193.8 MB


In [6]:
df_train_cleaned.describe()

,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory
count,6349896.0,6349896.0,6349896.0
mean,0.293805,0.242697,0.414767
std,0.336561,0.212787,0.218181
min,0.0,0.0,0.0002
25%,0.0,0.070864,0.2462
50%,0.201849,0.186062,0.4035
75%,0.566662,0.349793,0.5627
max,1.0,1.0,1.0


In [7]:
weight_n_clicks = 1
weight_time_on_page = 2
weight_scroll_percent = 2

df_train_cleaned['score'] = weight_n_clicks*df_train_cleaned['numberOfClicksHistory'] + weight_time_on_page*df_train_cleaned['timeOnPageHistory'] + weight_scroll_percent*df_train_cleaned['scrollPercentageHistory']
df_train_cleaned.describe()

,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,score
count,6349896.0,6349896.0,6349896.0,6349896.0
mean,0.293805,0.242697,0.414767,1.608734
std,0.336561,0.212787,0.218181,0.867295
min,0.0,0.0,0.0002,0.002373
25%,0.0,0.070864,0.2462,0.924991
50%,0.201849,0.186062,0.4035,1.56746
75%,0.566662,0.349793,0.5627,2.202917
max,1.0,1.0,1.0,4.990057


In [8]:
df_train_cleaned.drop(columns=["numberOfClicksHistory", "timeOnPageHistory", "scrollPercentageHistory"],inplace=True)

In [9]:
df_train_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6349896 entries, 0 to 6349895
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   userId    string  
 1   history   string  
 2   userType  category
 3   score     Float32 
dtypes: Float32(1), category(1), string(2)
memory usage: 133.2 MB


In [10]:
df_train_cleaned.describe()

,score
count,6349896.0
mean,1.608734
std,0.867295
min,0.002373
25%,0.924991
50%,1.56746
75%,2.202917
max,4.990057


In [11]:
df_train_cleaned.to_csv(config["DF_TRAIN_SCORES"])